In [165]:
%pylab inline
import xml.sax.saxutils as saxutils
import re
import pandas as pd
from bs4 import BeautifulSoup
import codecs


Populating the interactive namespace from numpy and matplotlib


In [166]:
#General constants 

# Newsline folder and format
data_folder = '/Users/xiuli/Downloads/reuters21578/' 

sgml_number_of_files = 22
sgml_file_name_template = 'reut2-NNN.sgm'

In [167]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
   
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
stop_words = set(stopwords.words('english'))
#stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

In [168]:
document_X = {}
document_Y = {}

document_id=[]
document_body=[]
document_train_test=[]
category=[]
document_X=pd.DataFrame()

for i in range(sgml_number_of_files):
    if i < 10:
        seq = '00' + str(i)
    else:
        seq = '0' + str(i)
        
    file_name = sgml_file_name_template.replace('NNN', seq)
#    print('Reading file: %s' % file_name)
    
    with codecs.open(data_folder + file_name, 'r', encoding='utf-8',
                 errors='ignore') as file:
        content = BeautifulSoup(file.read().lower())      
        for newsline in content('reuters'):
                      
            # News-line Id
            document_id.append(newsline['newid'])
            
            # News-line text
            document_body.append(unescape(strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')))
#            document_body.append(newsline.find("title").text)
            document_train_test.append(strip_tags(str(newsline.attrs["lewissplit"]) ))
            #for c in newsline.find("topics").findAll('d'):
                #category.append(c.text)
            category.append([item.text for item in newsline.find("topics").findAll('d')])
            
document_X['document_id'] = document_id
document_X['document_body'] = document_body
#document_X['document_body'] = document_X['document_body'].str.lower()
document_X['document_body'] = document_X['document_body'].apply(cleanHtml)
document_X['document_body'] = document_X['document_body'].apply(cleanPunc)
document_X['document_body'] = document_X['document_body'].apply(keepAlpha)
document_X['document_body'] = document_X['document_body'].apply(removeStopWords)
document_X['train_test_label'] = document_train_test
document_X['category'] = category

document_X.head()

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm
Reading file: reut2-021.sgm


In [169]:
train_X=document_X[(document_X['train_test_label']=='train') & (document_X['category'].astype(str)!='[]') & (document_X['document_body'].astype(str)!='None')]['document_body'].values.tolist()
test_X=document_X[(document_X['train_test_label']=='test') & (document_X['category'].astype(str)!='[]')& (document_X['document_body'].astype(str)!='None')]['document_body'].values.tolist()

train_Y=document_X[(document_X['train_test_label']=='train') & (document_X['category'].astype(str)!='[]')& (document_X['document_body'].astype(str)!='None')]['category'].values.tolist()
test_Y=document_X[(document_X['train_test_label']=='test') & (document_X['category'].astype(str)!='[]')& (document_X['document_body'].astype(str)!='None')]['category'].values.tolist()
#train_X=document_X[(document_X['train_test_label']=='train') & (document_X['category']!='[]')]['document_body'].to_frame()
#test_X=document_X[(document_X['train_test_label']=='test') & (document_X['category']!='[]')]['document_body'].to_frame()

#train_Y=document_X[(document_X['train_test_label']=='train') & (document_X['category']!='[]')]['category'].to_frame()
#test_Y=document_X[(document_X['train_test_label']=='test') & (document_X['category']!='[]')]['category'].to_frame()

In [170]:
def listTostring(x):
    Y=[]
    for i in range(len(x)):
        r=", ".join( e for e in x[i] )
        Y.append(r)
    return Y


from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(listTostring(train_Y))
y_test=vectorizer.transform(listTostring(test_Y))

In [171]:
length=[]
for i in range(len(train_Y)):
    l=len(train_X[i])
    length.append(l)
    print(i,l)
max_len=max(length)
print(max_len)

0 2190
1 650
2 938
3 275
4 1015
5 231
6 614
7 478
8 158
9 1113
10 402
11 137
12 948
13 286
14 280
15 452
16 70
17 284
18 195
19 423
20 341
21 486
22 427
23 1803
24 522
25 890
26 198
27 336
28 185
29 70
30 792
31 419
32 322
33 1000
34 221
35 169
36 314
37 276
38 467
39 74
40 442
41 412
42 371
43 425
44 365
45 195
46 63
47 61
48 161
49 74
50 192
51 296
52 1366
53 208
54 399
55 654
56 1025
57 110
58 65
59 1955
60 2870
61 389
62 360
63 471
64 564
65 302
66 468
67 199
68 418
69 189
70 703
71 189
72 493
73 1157
74 242
75 52
76 237
77 84
78 61
79 2002
80 782
81 182
82 381
83 172
84 75
85 469
86 630
87 310
88 196
89 63
90 207
91 92
92 57
93 236
94 173
95 242
96 1523
97 2345
98 1979
99 143
100 181
101 128
102 166
103 275
104 583
105 319
106 86
107 593
108 790
109 393
110 2063
111 150
112 271
113 2833
114 3501
115 410
116 437
117 304
118 764
119 540
120 926
121 680
122 1331
123 2016
124 412
125 872
126 2241
127 544
128 2123
129 2093
130 2173
131 406
132 728
133 1683
134 2487
135 1594
136 752
137

1286 336
1287 334
1288 1381
1289 585
1290 328
1291 281
1292 502
1293 254
1294 322
1295 312
1296 390
1297 371
1298 147
1299 403
1300 565
1301 239
1302 1234
1303 179
1304 872
1305 472
1306 152
1307 983
1308 363
1309 2278
1310 226
1311 264
1312 345
1313 1709
1314 444
1315 284
1316 689
1317 333
1318 165
1319 3015
1320 616
1321 518
1322 388
1323 312
1324 589
1325 1315
1326 1102
1327 815
1328 750
1329 289
1330 640
1331 390
1332 186
1333 130
1334 179
1335 85
1336 185
1337 472
1338 152
1339 687
1340 375
1341 828
1342 1299
1343 491
1344 845
1345 352
1346 67
1347 63
1348 67
1349 3178
1350 1162
1351 128
1352 70
1353 371
1354 1354
1355 665
1356 67
1357 451
1358 1587
1359 1995
1360 596
1361 504
1362 1263
1363 172
1364 1693
1365 890
1366 574
1367 237
1368 60
1369 454
1370 432
1371 333
1372 470
1373 766
1374 166
1375 80
1376 161
1377 421
1378 2021
1379 2813
1380 197
1381 91
1382 422
1383 216
1384 342
1385 212
1386 647
1387 63
1388 182
1389 584
1390 426
1391 352
1392 549
1393 354
1394 417
1395 1669
13

2535 184
2536 424
2537 149
2538 69
2539 246
2540 483
2541 1710
2542 327
2543 237
2544 1370
2545 4090
2546 1012
2547 193
2548 368
2549 332
2550 106
2551 441
2552 274
2553 337
2554 67
2555 187
2556 68
2557 117
2558 357
2559 189
2560 364
2561 769
2562 540
2563 379
2564 295
2565 60
2566 333
2567 228
2568 2469
2569 242
2570 855
2571 235
2572 721
2573 1155
2574 352
2575 996
2576 877
2577 348
2578 721
2579 573
2580 366
2581 2084
2582 356
2583 688
2584 1278
2585 2142
2586 338
2587 194
2588 1017
2589 326
2590 657
2591 808
2592 1387
2593 479
2594 764
2595 694
2596 1281
2597 864
2598 779
2599 2791
2600 1252
2601 2060
2602 3055
2603 1630
2604 474
2605 3108
2606 1406
2607 1404
2608 1219
2609 921
2610 694
2611 277
2612 932
2613 1508
2614 373
2615 1252
2616 1173
2617 344
2618 870
2619 807
2620 3173
2621 1223
2622 397
2623 2310
2624 438
2625 426
2626 1047
2627 340
2628 4755
2629 333
2630 740
2631 417
2632 493
2633 426
2634 437
2635 413
2636 976
2637 324
2638 411
2639 289
2640 1340
2641 1622
2642 374
2

3785 65
3786 67
3787 108
3788 896
3789 138
3790 79
3791 152
3792 2581
3793 392
3794 1954
3795 1434
3796 529
3797 843
3798 2590
3799 379
3800 493
3801 124
3802 327
3803 959
3804 65
3805 317
3806 929
3807 2653
3808 315
3809 151
3810 894
3811 953
3812 416
3813 229
3814 163
3815 1063
3816 219
3817 68
3818 224
3819 449
3820 171
3821 175
3822 1422
3823 3024
3824 541
3825 263
3826 741
3827 256
3828 152
3829 3156
3830 413
3831 130
3832 1964
3833 402
3834 471
3835 679
3836 66
3837 413
3838 267
3839 3158
3840 821
3841 580
3842 557
3843 245
3844 220
3845 490
3846 419
3847 1080
3848 140
3849 64
3850 372
3851 1384
3852 1155
3853 835
3854 1306
3855 432
3856 182
3857 287
3858 380
3859 365
3860 186
3861 177
3862 887
3863 899
3864 430
3865 426
3866 168
3867 292
3868 222
3869 479
3870 147
3871 483
3872 216
3873 239
3874 527
3875 129
3876 90
3877 340
3878 397
3879 1871
3880 58
3881 67
3882 377
3883 132
3884 1287
3885 594
3886 421
3887 207
3888 350
3889 83
3890 171
3891 403
3892 249
3893 290
3894 341
3895

5284 325
5285 57
5286 198
5287 333
5288 1665
5289 270
5290 380
5291 412
5292 255
5293 1514
5294 200
5295 200
5296 391
5297 241
5298 67
5299 1340
5300 912
5301 1458
5302 135
5303 437
5304 472
5305 249
5306 1139
5307 280
5308 455
5309 916
5310 944
5311 379
5312 961
5313 607
5314 221
5315 357
5316 154
5317 1997
5318 2026
5319 1170
5320 119
5321 117
5322 521
5323 77
5324 63
5325 128
5326 418
5327 2744
5328 426
5329 307
5330 2544
5331 472
5332 816
5333 196
5334 1418
5335 684
5336 3582
5337 930
5338 949
5339 395
5340 393
5341 744
5342 1067
5343 411
5344 2323
5345 1846
5346 1528
5347 645
5348 310
5349 907
5350 397
5351 3354
5352 420
5353 454
5354 2250
5355 926
5356 307
5357 366
5358 408
5359 757
5360 355
5361 364
5362 337
5363 797
5364 345
5365 218
5366 767
5367 910
5368 202
5369 588
5370 929
5371 1542
5372 799
5373 3777
5374 1219
5375 470
5376 548
5377 684
5378 479
5379 3218
5380 311
5381 524
5382 707
5383 420
5384 1304
5385 420
5386 1869
5387 380
5388 428
5389 1181
5390 2780
5391 1295
5392 

6284 354
6285 493
6286 462
6287 847
6288 773
6289 532
6290 400
6291 546
6292 434
6293 138
6294 64
6295 90
6296 114
6297 532
6298 767
6299 126
6300 531
6301 279
6302 221
6303 430
6304 394
6305 803
6306 1503
6307 218
6308 305
6309 218
6310 321
6311 194
6312 135
6313 314
6314 114
6315 237
6316 264
6317 96
6318 121
6319 605
6320 266
6321 62
6322 1361
6323 859
6324 154
6325 1162
6326 63
6327 131
6328 773
6329 285
6330 583
6331 3702
6332 862
6333 876
6334 1971
6335 353
6336 283
6337 185
6338 673
6339 161
6340 234
6341 116
6342 1537
6343 546
6344 2657
6345 684
6346 294
6347 92
6348 355
6349 500
6350 144
6351 235
6352 145
6353 589
6354 680
6355 183
6356 945
6357 336
6358 179
6359 227
6360 61
6361 58
6362 391
6363 118
6364 344
6365 610
6366 2979
6367 1122
6368 536
6369 311
6370 1801
6371 92
6372 334
6373 124
6374 2741
6375 3307
6376 143
6377 401
6378 140
6379 553
6380 202
6381 480
6382 133
6383 188
6384 867
6385 531
6386 224
6387 63
6388 113
6389 156
6390 111
6391 491
6392 364
6393 221
6394 369

In [172]:
new_list = []

for item in train_Y:
    for i in item:
        if i not in new_list:
            new_list.append(i)

print(len(new_list))
print(len(train_Y))

114
7068


In [173]:
from keras.preprocessing.text import Tokenizer
vect=Tokenizer()
vect.fit_on_texts(train_X)
vocab_size = len(vect.word_index) + 1
print(vocab_size)

22064


In [174]:
from keras.preprocessing.sequence import pad_sequences
#pd.Series(train_X)
encoded_docs_train = vect.texts_to_sequences(train_X)
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200, padding='post')
print(padded_docs_train)

[[4695  533 1703 ...    0    0    0]
 [  10  186  108 ...    0    0    0]
 [2015  202   79 ...    0    0    0]
 ...
 [  18  815    1 ...    0    0    0]
 [1359  713   61 ...    0    0    0]
 [ 736   21 1444 ...    0    0    0]]


In [175]:
encoded_docs_test =  vect.texts_to_sequences(test_X)
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200, padding='post')
#encoded_docs_cv = vect.texts_to_sequences(cv['preprocessed_plots'])
#padded_docs_cv = pad_sequences(encoded_docs_cv, maxlen=1200, padding='post')

In [180]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM,Embedding
from keras import utils, callbacks,layers, models 
from keras.preprocessing.text import Tokenizer

n_classes=187

model = Sequential()
# Configuring the parameters
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1200, 50)          1103200   
_________________________________________________________________
lstm_13 (LSTM)               (None, 1200, 128)         91648     
_________________________________________________________________
dropout_13 (Dropout)         (None, 1200, 128)         0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 187)               12155     
Total params: 1,256,411
Trainable params: 1,256,411
Non-trainable params: 0
_________________________________________________________________


In [181]:
model.compile(optimizer='adam', loss='binary_crossentropy')
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

Train on 6361 samples, validate on 707 samples
Epoch 1/5
6361/6361 [==============================] - 654s 103ms/step - loss: 0.1198 - val_loss: 0.0331
Epoch 2/5
6361/6361 [==============================] - 533s 84ms/step - loss: 0.0295 - val_loss: 0.0313
Epoch 3/5
6361/6361 [==============================] - 565s 89ms/step - loss: 0.0275 - val_loss: 0.0310
Epoch 4/5
6361/6361 [==============================] - 555s 87ms/step - loss: 0.0269 - val_loss: 0.0310
Epoch 5/5
6361/6361 [==============================] - 569s 89ms/step - loss: 0.0265 - val_loss: 0.0310


In [186]:
from sklearn.metrics import precision_score, recall_score, f1_score
predictions=model.predict([padded_docs_test])
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
    print("Micro-average quality numbers")
    print("thresholds",val)
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

#    precision1 = precision_score(y_test, pred, average='macro')
#    recall1 = recall_score(y_test, pred, average='macro')
#    f11 = f1_score(y_test, pred, average='macro')   

#    print("Macro-average quality numbers")
#    print("Precision1: {:.4f}, Recall1: {:.4f}, F1-measure1: {:.4f}".format(precision1, recall1, f11))

Micro-average quality numbers
thresholds 0.1
Precision: 0.3060, Recall: 0.4935, F1-measure: 0.3778
Micro-average quality numbers
thresholds 0.2
Precision: 0.3792, Recall: 0.3058, F1-measure: 0.3386
Micro-average quality numbers
thresholds 0.3
Precision: 0.3792, Recall: 0.3058, F1-measure: 0.3386
Micro-average quality numbers
thresholds 0.4
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
Micro-average quality numbers
thresholds 0.5
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
Micro-average quality numbers
thresholds 0.6
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
Micro-average quality numbers
thresholds 0.7
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
Micro-average quality numbers
thresholds 0.8
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
Micro-average quality numbers
thresholds 0.9
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
